## {nbdev} integration

These utilities are intended to be used to integrate nbdev into the workflow, alongside some opinionated tools like `{poetry}`, and also enforcing some of my personal defaults. 

In [ ]:
#| default_exp nbdev
#| export
import tomllib
from nbdev import nbdev_export, show_doc
from nbdev.config import nbdev_create_config, read_config_file
from pathlib import Path

In [ ]:
#| export
def export():
    "Syncs poetry config, nbdev's settings.ini, and does an export to the module."

    with open("pyproject.toml", "rb") as f:
        data = tomllib.load(f)["tool"]["poetry"]
    comment = "\n    # Set from pyproject.toml"
    authors = ", ".join(aut.split("<")[0].strip() for aut in data["authors"])
    poetry_config = {
        "repo": data["name"] + comment,
        "lib_name": data["name"] + comment,
        "lib_path": data["name"] + comment,
        "version": data["version"] + comment,
        "description": data["description"] + comment,
        "license": data["license"] + comment,
        "author": authors + comment,
        "author_email": ", ".join(
            aut.split("<")[1].replace(">", "") for aut in data["authors"]
        )
        + comment,
        "min_python": data["dependencies"]["python"].replace("^", "") + comment,
        "copyright": f"2022 onwards, {authors}" + comment
    }

    settings_file = Path("settings.ini")
    if settings_file.is_file():
        config = dict(read_config_file(settings_file)) | poetry_config
        settings_file.unlink()
    else:
        config = poetry_config
    nbdev_create_config(**config)
    nbdev_export(config["nbs_path"])
